In [5]:
from __future__ import print_function
from pyspark import SparkContext
from pyspark.mllib.classification import LogisticRegressionModel
if __name__ == "__main__":
   sc = SparkContext(appName="Pspark mllib Example")
   data = sc.textFile("test.data")
   ratings = data.map(lambda l: l.split(','))\
      .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
   
   # Build the recommendation model using Alternating Least Squares
   rank = 10
   numIterations = 10
   model = LogisticRegressionModel.train(ratings, rank, numIterations)
   
   # Evaluate the model on training data
   testdata = ratings.map(lambda p: (p[0], p[1]))
   predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
   ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
   MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
   print("Mean Squared Error = " + str(MSE))

AttributeError: type object 'LogisticRegressionModel' has no attribute 'train'

In [5]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating,LogisticRegressionWithSGD
data = sc.textFile("test.data")
ratings = data.map(lambda l: l.split(',')).map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
   
   # Build the recommendation model using Alternating Least Squares
rank = 10
numIterations = 10
model = LogisticRegressionWithSGD.train(ratings, rank, numIterations)
   
   # Evaluate the model on training data
testdata = ratings.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))

ImportError: cannot import name 'LogisticRegressionWithSGD' from 'pyspark.mllib.recommendation' (C:\Users\603766\AppData\Local\Continuum\anaconda3\lib\site-packages\pyspark\mllib\recommendation.py)

In [12]:
from pyspark.mllib.classification import LogisticRegressionWithSGD

In [42]:
data = sc.textFile("test.data")
ratings = data.map(lambda l: l.split(','))\
      .map(lambda l: LabeledPoint(int(l[0]), [int(l[1]), float(l[2])]))
ratings.collect() 


[LabeledPoint(1.0, [1.0,5.0]),
 LabeledPoint(1.0, [2.0,1.0]),
 LabeledPoint(1.0, [3.0,5.0]),
 LabeledPoint(1.0, [4.0,1.0]),
 LabeledPoint(2.0, [1.0,5.0]),
 LabeledPoint(2.0, [2.0,1.0]),
 LabeledPoint(2.0, [3.0,5.0]),
 LabeledPoint(2.0, [4.0,1.0]),
 LabeledPoint(3.0, [1.0,1.0]),
 LabeledPoint(3.0, [2.0,5.0]),
 LabeledPoint(3.0, [3.0,1.0]),
 LabeledPoint(3.0, [4.0,5.0]),
 LabeledPoint(4.0, [1.0,1.0]),
 LabeledPoint(4.0, [2.0,5.0]),
 LabeledPoint(4.0, [3.0,1.0]),
 LabeledPoint(4.0, [4.0,5.0])]

In [48]:
from pyspark.mllib.regression import LabeledPoint
data = [LabeledPoint(0.0, [0.0, 1.0]),LabeledPoint(1.0, [1.0, 0.0]),LabeledPoint(1.0, [1.0, 0.0])]
lrm = LogisticRegressionWithSGD.train(sc.parallelize(data), iterations=10)
lrm.predict([1.0, 0.0])
lrm.predict([0.0, 1.0])
lrm.predict(sc.parallelize([[1.0, 0.0], [0.0, 1.0]])).collect()
lrm.clearThreshold()
lrm.predict([0.0, 1.0])

0.335053006914351

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
sc= SparkContext()
sqlContext = SQLContext(sc)
house_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('test.data')
house_df.take(1)

[Row(col1=1, col2=1, label=5.0)]

In [3]:
house_df.cache()
house_df.printSchema()

root
 |-- col1: integer (nullable = true)
 |-- col2: integer (nullable = true)
 |-- label: double (nullable = true)



In [70]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['col1','col2'], outputCol = 'features')
vectorAssembler2 = VectorAssembler(inputCols = ['label'], outputCol = 'lab')
vhouse_df = vectorAssembler.transform(house_df)
vhouse_df = vhouse_df.select(['features', 'label'])
vhouse_df.show(3)

+---------+-----+
| features|label|
+---------+-----+
|[1.0,1.0]|  5.0|
|[1.0,2.0]|  1.0|
|[1.0,3.0]|  5.0|
+---------+-----+
only showing top 3 rows



In [6]:
splits = vhouse_df.randomSplit([0.8, 0.2])
train_df = splits[0]
test_df = splits[1]

In [13]:
splits[0].show(3)

+---------+-----+
| features|label|
+---------+-----+
|[1.0,2.0]|  1.0|
|[1.0,3.0]|  5.0|
|[1.0,4.0]|  1.0|
+---------+-----+
only showing top 3 rows



In [10]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='label')
lr_model = lr.fit(vhouse_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [-7.048480275391986e-16,-1.1000837891891595e-15]
Intercept: 3.0000000000000044


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ml-bank').getOrCreate()
df = spark.read.csv('test.data', header = True, inferSchema = True)
df.printSchema()

root
 |-- col1: integer (nullable = true)
 |-- col2: integer (nullable = true)
 |-- label: double (nullable = true)



In [5]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['col1','label'], outputCol = 'features')
vhouse_df = vectorAssembler.transform(house_df)
vhouse_df = vhouse_df.select(['features', 'col2'])
vhouse_df.show(3)

+---------+----+
| features|col2|
+---------+----+
|[1.0,5.0]|   1|
|[1.0,1.0]|   2|
|[1.0,5.0]|   3|
+---------+----+
only showing top 3 rows



In [6]:
splits = vhouse_df.randomSplit([0.5, 0.5])
train_df = splits[0]
test_df = splits[1]

In [8]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='col2')
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.7941176470588214,-0.2573529411764691]
Intercept: 1.4632352941176459


In [10]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol='col2')
lr_model = lr.fit(train_df)
#print("Coefficients: " + str(lr_model.coefficients))
#print("Intercept: " + str(lr_model.intercept))

In [19]:
pred=lr_model.transform(test_df)
pred.show(5)
preds=pred.toDF("features","label",'rawPrediction','probability','prediction')

+---------+----+--------------------+--------------------+----------+
| features|col2|       rawPrediction|         probability|prediction|
+---------+----+--------------------+--------------------+----------+
|[1.0,1.0]|   4|[-36.997502620713...|[4.02072145144986...|       2.0|
|[1.0,5.0]|   3|[-96.225140212277...|[1.02360832696597...|       1.0|
|[2.0,1.0]|   2|[-49.016554081959...|[3.63127184570355...|       2.0|
|[2.0,1.0]|   4|[-49.016554081959...|[3.63127184570355...|       2.0|
|[2.0,5.0]|   3|[-108.24419167352...|[1.86768992343026...|       1.0|
+---------+----+--------------------+--------------------+----------+
only showing top 5 rows



In [17]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(preds)

0.044444444444444446

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ml-bank').getOrCreate()
df = spark.read.csv('bank.csv', header = True, inferSchema = True)
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



In [2]:
df = df.select('age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'deposit')
cols = df.columns
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



In [3]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
categoricalColumns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
label_stringIdx = StringIndexer(inputCol = 'deposit', outputCol = 'label')
stages += [label_stringIdx]
numericCols = ['age', 'balance', 'duration', 'campaign', 'pdays', 'previous']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [19]:
stages

[StringIndexer_fbe265a91a2b,
 OneHotEncoderEstimator_b3c4344881f5,
 StringIndexer_4f25b4c715e7,
 OneHotEncoderEstimator_8be2f6d5eb33,
 StringIndexer_21bffdcf5879,
 OneHotEncoderEstimator_270355f8c77f,
 StringIndexer_4938be76b41a,
 OneHotEncoderEstimator_cb6a9967797f,
 StringIndexer_1792b76ebddd,
 OneHotEncoderEstimator_5032b6766847,
 StringIndexer_585fd5606cf8,
 OneHotEncoderEstimator_24d56011de3d,
 StringIndexer_e7263604eeea,
 OneHotEncoderEstimator_af48fbe2613c,
 StringIndexer_06efc19b8af6,
 OneHotEncoderEstimator_aecbc339249a,
 StringIndexer_ee91bcdab23a,
 VectorAssembler_dedb9823f378]

In [4]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df)
df = pipelineModel.transform(df)
selectedCols = ['label', 'features'] + cols
df = df.select(selectedCols)
df.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



In [5]:
df.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



In [6]:
train, test = df.randomSplit([0.7, 0.3], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 7764
Test Dataset Count: 3398


In [7]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(train)

In [10]:
predictions = lrModel.transform(test)
predictions.select('age', 'job', 'label', 'rawPrediction', 'prediction', 'probability').show(10)

+---+----------+-----+--------------------+----------+--------------------+
|age|       job|label|       rawPrediction|prediction|         probability|
+---+----------+-----+--------------------+----------+--------------------+
| 37|management|  0.0|[1.19871810716723...|       0.0|[0.76829666339830...|
| 40|management|  0.0|[2.20534940465796...|       0.0|[0.90072886169926...|
| 53|management|  0.0|[1.02590348276690...|       0.0|[0.73612093009497...|
| 32|management|  0.0|[1.25795481657702...|       0.0|[0.77867383994058...|
| 54|management|  0.0|[1.33232096924268...|       0.0|[0.79122429116078...|
| 40|management|  0.0|[1.57095096412779...|       0.0|[0.82791913346617...|
| 56|management|  0.0|[3.06095963426752...|       0.0|[0.95525333386804...|
| 50|management|  0.0|[-0.8102603273804...|       1.0|[0.30783502428597...|
| 47|management|  0.0|[0.67024288891379...|       0.0|[0.66155754396054...|
| 44|management|  0.0|[1.29756265761715...|       0.0|[0.78542449653716...|
+---+-------

In [11]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()
print('Test Area Under ROC', evaluator.evaluate(predictions))

Test Area Under ROC 0.8858324614449619


In [12]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
dtModel = dt.fit(train)
predictions = dtModel.transform(test)
predictions.select('age', 'job', 'label', 'rawPrediction', 'prediction', 'probability').show(10)

+---+----------+-----+--------------+----------+--------------------+
|age|       job|label| rawPrediction|prediction|         probability|
+---+----------+-----+--------------+----------+--------------------+
| 37|management|  0.0|[2463.0,473.0]|       0.0|[0.83889645776566...|
| 40|management|  0.0|[2463.0,473.0]|       0.0|[0.83889645776566...|
| 53|management|  0.0|[2463.0,473.0]|       0.0|[0.83889645776566...|
| 32|management|  0.0|[2463.0,473.0]|       0.0|[0.83889645776566...|
| 54|management|  0.0|[2463.0,473.0]|       0.0|[0.83889645776566...|
| 40|management|  0.0|  [373.0,30.0]|       0.0|[0.92555831265508...|
| 56|management|  0.0|[2463.0,473.0]|       0.0|[0.83889645776566...|
| 50|management|  0.0|[788.0,1230.0]|       1.0|[0.39048562933597...|
| 47|management|  0.0|[788.0,1230.0]|       1.0|[0.39048562933597...|
| 44|management|  0.0|[2463.0,473.0]|       0.0|[0.83889645776566...|
+---+----------+-----+--------------+----------+--------------------+
only showing top 10 

In [13]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')
rfModel = rf.fit(train)
predictions = rfModel.transform(test)
predictions.select('age', 'job', 'label', 'rawPrediction', 'prediction', 'probability').show(10)

+---+----------+-----+--------------------+----------+--------------------+
|age|       job|label|       rawPrediction|prediction|         probability|
+---+----------+-----+--------------------+----------+--------------------+
| 37|management|  0.0|[14.7001710468374...|       0.0|[0.73500855234187...|
| 40|management|  0.0|[15.6431397667577...|       0.0|[0.78215698833788...|
| 53|management|  0.0|[13.3954437117651...|       0.0|[0.66977218558825...|
| 32|management|  0.0|[14.9205800987001...|       0.0|[0.74602900493500...|
| 54|management|  0.0|[15.1701520876645...|       0.0|[0.75850760438322...|
| 40|management|  0.0|[15.1441990994691...|       0.0|[0.75720995497345...|
| 56|management|  0.0|[16.7466741662731...|       0.0|[0.83733370831365...|
| 50|management|  0.0|[5.07354501572884...|       1.0|[0.25367725078644...|
| 47|management|  0.0|[9.90639420505103...|       1.0|[0.49531971025255...|
| 44|management|  0.0|[12.1156749922266...|       0.0|[0.60578374961133...|
+---+-------

In [14]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(maxIter=10)
gbtModel = gbt.fit(train)
predictions = gbtModel.transform(test)
predictions.select('age', 'job', 'label', 'rawPrediction', 'prediction', 'probability').show(10)

+---+----------+-----+--------------------+----------+--------------------+
|age|       job|label|       rawPrediction|prediction|         probability|
+---+----------+-----+--------------------+----------+--------------------+
| 37|management|  0.0|[0.57808138910181...|       0.0|[0.76063477260811...|
| 40|management|  0.0|[1.37467582901950...|       0.0|[0.93987672346171...|
| 53|management|  0.0|[-0.0012929624008...|       1.0|[0.49935351915983...|
| 32|management|  0.0|[0.61900313605401...|       0.0|[0.77521678642033...|
| 54|management|  0.0|[0.98157815641818...|       0.0|[0.87687413211579...|
| 40|management|  0.0|[0.96138354833170...|       0.0|[0.87244668327834...|
| 56|management|  0.0|[1.39120025731353...|       0.0|[0.94171733839668...|
| 50|management|  0.0|[-0.6141629093446...|       1.0|[0.22647458093662...|
| 47|management|  0.0|[-0.0439971283470...|       1.0|[0.47801561939801...|
| 44|management|  0.0|[0.26452511568224...|       0.0|[0.62926156628314...|
+---+-------

In [15]:
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

Test Area Under ROC: 0.8940728473145346


In [16]:
print(gbt.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 

In [17]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
paramGrid = (ParamGridBuilder()
             .addGrid(gbt.maxDepth, [2, 4, 6])
             .addGrid(gbt.maxBins, [20, 60])
             .addGrid(gbt.maxIter, [10, 20])
             .build())
cv = CrossValidator(estimator=gbt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
# Run cross validations.  This can take about 6 minutes since it is training over 20 trees!
cvModel = cv.fit(train)
predictions = cvModel.transform(test)
evaluator.evaluate(predictions)

KeyboardInterrupt: 